In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import collections
import matplotlib.pyplot as plt
import json

In [2]:
def create_watcher_obj(): #From the riotwater module, creates a wrapper watcher object that can access riot api
    api_key = get_api_key()
    watcher = LolWatcher(api_key)
    return watcher

In [3]:
def get_api_key(): #Called inside of "create_watcher_obj()" and simply reads the riot api key text file

    api_file = open("../riot_api_key.txt","r")
    api_key = api_file.read()
    api_file.close()
    return api_key

def get_acc_id(region='na1', summoner_name='we and i'): #Called inside of "get_match_list(x,y)"
     
    watcher_summoner = watcher.summoner.by_name(region,summoner_name)
    return watcher_summoner['accountId']

def get_match_list(region='na1', summoner_name="we and i"): #Returns an account's list of matches
    
    acc_id = get_acc_id(region, summoner_name)
    my_matches = watcher.match.matchlist_by_account(region, acc_id)
    return my_matches['matches']

def get_game_id_list(matchlist): #Given a matchlist, returns the game id of every match
    
    game_id_list = []
    for match in matchlist:
        game_id_list.append(match['gameId'])
    return game_id_list
        
def get_match_detail(game_id, region ='na1'): #Returns the summary details of a match
    
    match_detail = watcher.match.by_id(region, game_id)
    return match_detail

def get_match_timeline(game_id, region='na1'): #Returns a match's timeline object
    
    timeline = watcher.match.timeline_by_match(region, game_id)
    return timeline['frames'] #The other key in the timeline object is 'frameInterval' which I deemed unimportant

In [4]:
def jsonify_info(info):
    
    json_object = json.dumps(info, indent = 4)
    return json_object

In [7]:
watcher = create_watcher_obj()

In [40]:
def get_relev_timeline_info(timeline): #Called inside of 'get_relev_info(x,y)'
    
    a_dict = collections.defaultdict(list)
    
    for frame in timeline:
        temp_list = []
        for participant in frame['participantFrames']:
            relev_keys = ['participantId', 'position', 'currentGold', 
                          'totalGold', 'level', 'xp', 'minionsKilled', 'jungleMinionsKilled']
            
            temp_dict = {x: frame['participantFrames'][participant][x] for x in relev_keys if x in frame['participantFrames'][participant]}
            packaged_temp_dict = {participant:temp_dict}
            temp_list.append(packaged_temp_dict)
        a_dict[frame['timestamp']] = temp_list    
        
    return a_dict

def get_relev_event_info(timeline): #Called inside of 'get_relev_info(x,y)'
    
    a_dict = collections.defaultdict(list)
    i = 0
    for frame in timeline:
        temp_list = []
        for event in frame['events']:
            a_dict[i] = event
            i = i + 1
            
    return a_dict
            
           
def get_relev_summary_info(match_detail): #Called inside of 'get_relev_info(x,y)'
    
    a_dict = collections.defaultdict(list)
    
    game_keys = ['gameId', 'platformId', 'gameDuration', 'queueId', 'mapId', 
                 'seasonId', 'gameVersion', 'gameMode', 'gameType']
    team_keys = ['teams']
    player_keys = ['participants', 'participantIdentities']
    
    temp_game_dict = {x: match_detail[x] for x in game_keys if x in match_detail }
    temp_team_dict = {x: match_detail[x] for x in team_keys if x in match_detail }
    temp_player_dict = {x: match_detail[x] for x in player_keys if x in match_detail }
    
    a_dict['Game'] = temp_game_dict
    a_dict['Team'] = temp_team_dict
    a_dict['Player'] = temp_player_dict
    
    return a_dict          
            

In [41]:
def get_relev_info(timeline, match_detail):
    
    a_dict = collections.defaultdict(list)
    
    timeline_dict = get_relev_timeline_info(timeline)
    event_dict = get_relev_event_info(timeline)
    summary_dict = get_relev_summary_info(match_detail)
    
    a_dict['Timestamps'] = timeline_dict
    a_dict['Events'] = event_dict
    a_dict['Summary'] = summary_dict
    
    return a_dict

In [43]:
info = get_relev_info(time, match_det)
info_json = jsonify_info(info)
print(info_json)

{
    "Timestamps": {
        "0": [
            {
                "1": {
                    "participantId": 1,
                    "position": {
                        "x": 554,
                        "y": 581
                    },
                    "currentGold": 500,
                    "totalGold": 500,
                    "level": 1,
                    "xp": 0,
                    "minionsKilled": 0,
                    "jungleMinionsKilled": 0
                }
            },
            {
                "2": {
                    "participantId": 2,
                    "position": {
                        "x": 557,
                        "y": 345
                    },
                    "currentGold": 500,
                    "totalGold": 500,
                    "level": 1,
                    "xp": 0,
                    "minionsKilled": 0,
                    "jungleMinionsKilled": 0
                }
            },
            {
                "3": {
          

In [39]:
match_det = get_match_detail(game_id_example)
data_struc_summary = get_relev_summary_info(match_det)
data_struc_summary_json = jsonify_info(data_struc_summary)
print(data_struc_summary_json)

{
    "Game": {
        "gameId": 3935988097,
        "platformId": "NA1",
        "gameDuration": 1257,
        "queueId": 430,
        "mapId": 11,
        "seasonId": 13,
        "gameVersion": "11.11.377.6311",
        "gameMode": "CLASSIC",
        "gameType": "MATCHED_GAME"
    },
    "Team": {
        "teams": [
            {
                "teamId": 100,
                "win": "Win",
                "firstBlood": true,
                "firstTower": false,
                "firstInhibitor": false,
                "firstBaron": false,
                "firstDragon": true,
                "firstRiftHerald": false,
                "towerKills": 5,
                "inhibitorKills": 0,
                "baronKills": 0,
                "dragonKills": 2,
                "vilemawKills": 0,
                "riftHeraldKills": 0,
                "dominionVictoryScore": 0,
                "bans": []
            },
            {
                "teamId": 200,
                "win": "Fail",
   

In [16]:
match_list = get_match_list()
game_id_list = get_game_id_list(match_list)
game_id_example = game_id_list[0]

time = get_match_timeline(game_id_example)

In [24]:
data_struc_events = get_relev_event_info(time)
data_struc_events_json = jsonify_info(data_struc_events)
#print(data_struc_events_json)
print(data_struc_events.keys())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [11]:
data_struc_timeline = get_relev_timeline_info(time)
data_struc_timeline_json = jsonify_info(data_struc_timeline)
print(data_struc_timeline_json)

{
    "0": [
        {
            "1": {
                "participantId": 1,
                "position": {
                    "x": 554,
                    "y": 581
                },
                "currentGold": 500,
                "totalGold": 500,
                "level": 1,
                "xp": 0,
                "minionsKilled": 0,
                "jungleMinionsKilled": 0
            }
        },
        {
            "2": {
                "participantId": 2,
                "position": {
                    "x": 557,
                    "y": 345
                },
                "currentGold": 500,
                "totalGold": 500,
                "level": 1,
                "xp": 0,
                "minionsKilled": 0,
                "jungleMinionsKilled": 0
            }
        },
        {
            "3": {
                "participantId": 3,
                "position": {
                    "x": 335,
                    "y": 269
                },
                "c

In [ ]:
#NEXT, I WANT TO GET SOME OF THIS INTO VISUAL STUDIO CODE AND GET A REST API GOING AND THEN CONNECT THAT TO REACT AND THEN
#MAKE SOME SIMPLE GRAPHS THAT WILL DISPLAY ON A WEBPAGE. NOTHING FANCY, BUT AT THIS POINT I WANT TO TRY CONNECTING MY BACK
#TO FRONT END LETS GO :D